In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import random
import lr_scheduler
import func

folder_path = "F:/code/barlow/UR5"
file_pairs = func.load_file_pairs(folder_path)

AUTO = tf.data.AUTOTUNE
CROP_TO = 32
SEED = 200

PROJECT_DIM = 2048
BATCH_SIZE = 32
EPOCHS = 50

In [2]:
random_500_similar, random_500_dissimilar, remaining_similar, remaining_dissimilar = func.split_and_shuffle_pairs(file_pairs, folder_path)

def process_data(data_list):
    return [
        [func.read_jpg_files(jpg_path), func.read_and_parse_npy_file(npy_path)]
        for jpg_path, npy_path in data_list
    ]

xtrain1 = process_data(random_500_similar)
xtrain2 = process_data(random_500_dissimilar)
xtest1 = process_data(remaining_similar)
xtest2 = process_data(remaining_dissimilar)

# label 1 - similar, 0 - disimilar
ytrain1 = [1] * 500
ytrain2 = [0] * 500
ytest1 = [1] *79
ytest2 = [0] *79

y_train = ytrain1 + ytrain2
y_test = ytest1 + ytest2
x_train = xtrain1 + xtrain2
x_test = xtest1 + xtest2

In [3]:
def preprocess_data(data):
    images = []
    numerical_features = []
    for image, features in data:
        # flat
        images.append(image.flatten())
        numerical_features.append(features)
    
    images = np.array(images)
    numerical_features = np.array(numerical_features)
    
    combined_features = np.hstack((images, numerical_features))
    return combined_features


X_train = preprocess_data(x_train)
X_test = preprocess_data(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [4]:
def create_mlp_encoder(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(512, activation="relu")(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dense(128, activation=None)(x)  # Projection head
    return tf.keras.Model(inputs, x)

# Tạo encoder mới
mlp_encoder = create_mlp_encoder(input_dim=16392)
# Sử dụng encoder MLP cho Barlow Twins
barlow_twins = func.BarlowTwins(encoder=mlp_encoder)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9)
barlow_twins.compile(optimizer=optimizer)
# barlow_twins.load_weights('bl.weights.h5')

In [5]:
backbone = tf.keras.Model(
    inputs=barlow_twins.encoder.input,
    outputs=barlow_twins.encoder.layers[-1].output  # output từ lớp Dense(128)
)
# Freeze 
backbone.trainable = False

# add classifier
inputs = tf.keras.layers.Input(shape=(16392,))
x = backbone(inputs, training=False)

#outputs = tf.keras.layers.Dense(10, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="relu")(x)

test_model = tf.keras.Model(inputs, outputs, name="test_model")


In [6]:
# Shuffle train_ds vs test_ds

seed = 226
train_ds = X_train.shuffle(buffer_size=10, seed=seed, reshuffle_each_iteration=True)
test_ds = X_test.shuffle(buffer_size=10, seed=seed, reshuffle_each_iteration=True)


AttributeError: 'numpy.ndarray' object has no attribute 'shuffle'

In [ ]:
# Standard Scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [7]:
# Compile model
test_model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.00005, momentum=0.9)
)

history = test_model.fit(
    train_ds, validation_data=test_ds, epochs=100
)

_, test_acc = test_model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))

NameError: name 'train_ds' is not defined